<a href="https://colab.research.google.com/github/choe1842/ai-security/blob/master/Choejaegeon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Gradient Descent 구현 하기

import torch 
from torch.autograd import Variable

x_data = [1.0, 2.0, 3.0]
y_data = [3.0, 6.0, 9.0]

w = 1 # 임의의 값 지정

def forward(x): 
  return x * w # 기본 식

def loss(x,y): # loss 계산
  y_pred = forward(x)
  return (y_pred - y) * (y_pred - y) # 예측 값과 실제 값의 거리 차를 돌려줌

def gradient(x,y): 
  return 2 * x * (x * w - y) # gradient를 계산함

print("학습 전", "값 :", forward(4))

for epoch in range(15): # 학습 과정
  for x_val, y_val in zip(x_data, y_data):
    grad = gradient(x_val, y_val)
    
    w = w - 0.01 * grad # 가중치 업데이트
    
    print("\tgrad:", x_val, y_val, round(grad,2))
    l = loss(x_val, y_val)
   
  print("prograss:", epoch, "w = ", round(w,2), "loss = ", round(l,5))
  
 
print("학습완료", "값 :", forward(4)) # 학습 확인


# Advanced Convolutional neural network 구현하기


import torch 
import torch.nn as nn # 딥러닝 모델에 필요한 모듈이 모아져있는 패키지
import torch.nn.functional as F # 함수의 input으로 반드시 연산되어야 하는 값을 받는 기능
import torch.optim as optim # 최적화 방법이 있는 패키지
from torchvision import datasets, transforms # torch에서 제공해주는 데이터셋 가져오기

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # gpu 사용 가능 확인
batch_size = 64

train_dataset = datasets.MNIST(root='./mnist_data/',train = True, transform = transforms.ToTensor(),download = True) # 학습용 데이터셋

test_dataset = datasets.MNIST(root='./mnist_data/',train = False, transform = transforms.ToTensor()) # 학습 정도를 테스트하기 위한 데이터셋

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)  # 학습 데이터를 받아오는 함수

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False) # 테스트 데이터를 받아오는 함수

class InceptionModule(nn.Module): # 여러 필터의 결과를 합치기 위한 인셉션 모듈 구현, nn.Module 상속

  def __init__(self,in_channels): # 네트워크 계층 정의
    super(InceptionModule,self).__init__() 
    self.branch1x1 = nn.Conv2d(in_channels,16,kernel_size=1) # _1 : 연산을 줄이면서 한 레이어에서 더 깊은 논리 처리 가능
  
    self.branch5x5_1 = nn.Conv2d(in_channels,16,kernel_size=1) # _1 과 같은 기능 
    self.branch5x5_2 = nn.Conv2d(16,24,kernel_size=5,padding=2) # 16채널을 5X5의 24개 채널로 만듦, 패딩을 통해 결과값이 작아지는 것을 방지
  
    self.branch3x3_1 = nn.Conv2d(in_channels,16,kernel_size=1) # _1 과 같은 기능
    self.branch3x3_2 = nn.Conv2d(16,24,kernel_size=3,padding=1) # 16채널을 3X3의 24개 채널로 만듦, 패딩을 통해 결과값이 작아지는 것을 방지
    self.branch3x3_3 = nn.Conv2d(24,24,kernel_size=3,padding=1) # 24채널을 3X3의 24개 채널로 만듦, 패딩을 통해 결과값이 작아지는 것을 방지
  
    self.branch_pool = nn.Conv2d(in_channels,24,kernel_size=1) 

  def forward(self,x): # 하나의 x를 4개로 쪼개주기 위함
    
    branch1x1 = self.branch1x1(x) 
    
    branch5x5 = self.branch5x5_1(x) 
    branch5x5 = self.branch5x5_2(branch5x5) 
    
    branch3x3 = self.branch3x3_1(x) 
    branch3x3 = self.branch3x3_2(branch3x3) 
    branch3x3 = self.branch3x3_3(branch3x3) 
    
    branch_pool = F.avg_pool2d(x,kernel_size=3,stride=1,padding=1) 
    branch_pool = self.branch_pool(branch_pool) 
    
    outputs = [branch1x1,branch5x5,branch3x3,branch_pool]
    return torch.cat(outputs, 1) # 4개의 함수를 하나로 합쳐준다. 채널의 개수는 총 88개(16+24+24+24)
  
  
  
class MainNet(nn.Module): # 전체 모델 구성
  def __init__(self): 
    super(MainNet,self).__init__() 
    self.conv1 = nn.Conv2d(1,10,kernel_size=5)  # 5x5의 Conv layer 10채널 통과
    
    self.conv2 = nn.Conv2d(88,20,kernel_size=5) # # 5x5의 Conv layer 20채널 통과, 인셉션 모델의 채널의 개수가 총 88개 이기 때문에 채널을 88개로 함
    
    self.incept1 = InceptionModule(in_channels=10) # 인셉션 모듈 통과
    self.incept2 = InceptionModule(in_channels=20) # 인셉션 모듈 통과
    
    self.max_pool = nn.MaxPool2d(2)  # 주어진 필터 범위 내에서 최대값을 뽑아내는 과정
    self.fc = nn.Linear(1408,10) # 마지막 Conv layer의 cell을 한줄로 나열한 후 그 개수에 맞추어 Fully connected layer의 input으로 넘겨주는 것, 즉, 1408개의 데이터를 10개로 바꿈

  def forward(self,x): 
    in_size = x.size(0) # size(0)을 하면 (n, 28*28)중 n을 리턴함 
    x = F.relu(self.max_pool(self.conv1(x))) # 학습을 위한 활성화 함수 relu
    x = self.incept1(x) 
    x = F.relu(self.max_pool(self.conv2(x))) 
    x = self.incept2(x) 
    x = x.view(in_size,-1) 
    x = self.fc(x) 
    return F.log_softmax(x) # 아웃풋을 한번에 묶어 결과를 조정해주는 소프트맥스 함수 사용

model = MainNet().to(device) # 모델 정의
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5) # 가중치를 업데이트 시켜주는 최적화 (Stochastic Gradient Descent) 사용, 과도한 연산 방지
criterion = nn.NLLLoss() # loss 함수 정의, softmax함수를 사전에 사용했기 때문에, NLLLoss를 사용합니다.

def train(epoch): 
  model.train() 
  for batch_idx, (data, target) in enumerate(train_loader): 
    data = data.to(device) 
    target = target.to(device) 
    
    output = model(data) # 데이터를 학습 모델에 투입
    
    optimizer.zero_grad() # gradient buffers를 0으로
    loss = criterion(output, target) # loss 계산
    loss.backward()  # 역전파 과정을 통해 각 변수마다 loss에 대한 gradient를 구해줌
    optimizer.step() # model의 파라미터들을 업데이트 해주는 함수
    if batch_idx % 50 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.data))

#batch: 전체 데이터를 나눠 일부를 묶은 것을 의미함
#epoch: 모든 학습 데이터에 대해 forward 와 backward pass를 한번 진행한 상태

def test(): # 과학습 가능성을 고려하여 별도의 테스트 함수 선언
  model.eval() 
  test_loss = 0 
  correct = 0 
  for data, target in test_loader: # 테스트 데이터 사용
    data = data.to(device) 
    target = target.to(device) 
    output = model(data) 
    test_loss += criterion(output, target).data # loss 계산
    pred = output.data.max(1, keepdim=True)[1] 
    correct += pred.eq(target.data.view_as(pred)).cpu().sum() #pred배열과 data의 일치 여부 검사해주는 기능
   
  test_loss /= len(test_loader.dataset)/batch_size 
  print('\nTest set: 평균 loss: {:.4f}, 정확도: {}/{} ({:.0f}%) \n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10): # 최종 실행
  train(epoch)
  test()
    
    
    
    
    

학습 전 값 : 4
	grad: 1.0 3.0 -4.0
	grad: 2.0 6.0 -15.68
	grad: 3.0 9.0 -32.46
prograss: 0 w =  1.52 loss =  19.67696
	grad: 1.0 3.0 -2.96
	grad: 2.0 6.0 -11.59
	grad: 3.0 9.0 -24.0
prograss: 1 w =  1.91 loss =  10.75508
	grad: 1.0 3.0 -2.19
	grad: 2.0 6.0 -8.57
	grad: 3.0 9.0 -17.74
prograss: 2 w =  2.19 loss =  5.87853
	grad: 1.0 3.0 -1.62
	grad: 2.0 6.0 -6.34
	grad: 3.0 9.0 -13.12
prograss: 3 w =  2.4 loss =  3.2131
	grad: 1.0 3.0 -1.2
	grad: 2.0 6.0 -4.68
	grad: 3.0 9.0 -9.7
prograss: 4 w =  2.56 loss =  1.75622
	grad: 1.0 3.0 -0.88
	grad: 2.0 6.0 -3.46
	grad: 3.0 9.0 -7.17
prograss: 5 w =  2.67 loss =  0.95992
	grad: 1.0 3.0 -0.65
	grad: 2.0 6.0 -2.56
	grad: 3.0 9.0 -5.3
prograss: 6 w =  2.76 loss =  0.52468
	grad: 1.0 3.0 -0.48
	grad: 2.0 6.0 -1.89
	grad: 3.0 9.0 -3.92
prograss: 7 w =  2.82 loss =  0.28678
	grad: 1.0 3.0 -0.36
	grad: 2.0 6.0 -1.4
	grad: 3.0 9.0 -2.9
prograss: 8 w =  2.87 loss =  0.15675
	grad: 1.0 3.0 -0.26
	grad: 2.0 6.0 -1.03
	grad: 3.0 9.0 -2.14
prograss: 9 w =  2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.300515
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.283001
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.157422
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.918620
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.596030
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.520006
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.427651
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.386381
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.305072
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.288072
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.104975
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.252903
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.232002
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.130033
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.366724
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.197367
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.347452
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.109280

Test set: 평균 loss: 0.1597, 정확도: 9513/10000 (95%) 

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.227709
Train E